In [13]:
import pyomo.environ as pyo
from idaes.core import FlowsheetBlock
from idaes.models.properties.modular_properties.base.generic_property import (
    GenericParameterBlock)
import idaes.core.util.scaling as iscale
from idaes.models_extra.power_generation.properties.natural_gas_PR import (
    get_prop,
    EosType,
)
from idaes.core.util.model_statistics import (report_statistics,
                                              unfixed_variables_set,
                                              degrees_of_freedom)
from idaes_ui.fv import visualize 

In [14]:
import importlib

from SOFC_unit import SofcUnit

def sofc_example_flowsheet(eos=EosType.PR):
    m = pyo.ConcreteModel()
    m.fs = FlowsheetBlock(dynamic=False)

    # --- property packages ---
    fuel_comps      = {"H2": 0.99, "H2O": 0.01}
    air_comps       = {"O2": 0.21, "N2": 0.79}

    m.fs.cathode_side_prop_package = GenericParameterBlock(
        **get_prop(air_comps, {"Vap"}, eos=eos),
        doc="Air property parameters",
    )
    m.fs.anode_side_prop_package = GenericParameterBlock(
        **get_prop(fuel_comps, {"Vap"}, eos=eos),
        doc="Flue gas property parameters",
    )
    m.fs.sofc = SofcUnit(
        cathode_side_prop_package=m.fs.cathode_side_prop_package,
        anode_side_prop_package=m.fs.anode_side_prop_package,
        reaction_eos=eos
    )


    # design / operating points
    m.fs.sofc.fuel_util.fix(0.7)
    m.fs.sofc.number_of_cells.fix(80)
    m.fs.sofc.cell_area.fix(0.320)

    # 1) H₂ translator inlet
    m.fs.sofc.anode_side_inlet.flow_mol[0].fix(0.075397)
    m.fs.sofc.anode_side_inlet.temperature[0].fix(942.05)
    m.fs.sofc.anode_side_inlet.pressure[0].fix(101_325)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "H2"].fix(0.99)
    m.fs.sofc.anode_side_inlet.mole_frac_comp[0, "H2O"].fix(0.01)

    # 2) Air separator inlet
    m.fs.sofc.cathode_side_inlet.flow_mol[0].fix(2)
    m.fs.sofc.cathode_side_inlet.temperature[0].fix(948.45)
    m.fs.sofc.cathode_side_inlet.pressure[0].fix(101_325)
    m.fs.sofc.cathode_side_inlet.mole_frac_comp[0, "O2"].fix(0.21)
    m.fs.sofc.cathode_side_inlet.mole_frac_comp[0, "N2"].fix(0.79)
    
    
    return m

# create a flowsheet with the sofc model
model = sofc_example_flowsheet()


ERROR: Rule failed when generating expression for Constraint
fs.sofc.sofc_energy_balance with index 0.0: AttributeError:
'_ScalarStoichiometricReactor' object has no attribute 'heat_duty'
ERROR: Constructing component 'fs.sofc.sofc_energy_balance' from data=None
failed:
        AttributeError: '_ScalarStoichiometricReactor' object has no attribute
        'heat_duty'
2025-06-09 12:32:31 [ERROR] idaes.core.base.process_block: Failure in build: fs.sofc
Traceback (most recent call last):
  File "c:\Users\Sara\miniforge3\envs\my-idaes-env\lib\site-packages\idaes\core\base\process_block.py", line 41, in _rule_default
    b.build()
  File "c:\Users\Sara\Desktop\IDAES\SOFC\SOFC_unit.py", line 112, in build
    self._add_constraints()
  File "c:\Users\Sara\Desktop\IDAES\SOFC\SOFC_unit.py", line 339, in _add_constraints
    def sofc_energy_balance(b, t):
  File "c:\Users\Sara\miniforge3\envs\my-idaes-env\lib\site-packages\pyomo\core\base\block.py", line 77, in __call__
    setattr(
  File "c:\U

AttributeError: '_ScalarStoichiometricReactor' object has no attribute 'heat_duty'

In [ ]:
# from contextlib import contextmanager
# from idaes.core.util.model_statistics import degrees_of_freedom
# from pyomo.environ import value, Var

# @contextmanager
# def _temp_fix(v):
#     """Context-manager: fissa v al suo valore, poi lo ri-sblocca"""
#     was_fixed = v.fixed
#     if not was_fixed:
#         v.fix(value(v))
#     try:
#         yield
#     finally:
#         if not was_fixed:
#             v.unfix()

# def find_dof_variables(mdl, max_hits=10000):
#     """Restituisce le prime variabili che contano nei DoF"""
#     dof0 = degrees_of_freedom(mdl)
#     hot = []
#     for var in mdl.component_data_objects(Var, descend_into=True):
#         if var.fixed:
#             continue
#         with _temp_fix(var):
#             if degrees_of_freedom(mdl) == dof0 - 1:
#                 hot.append(var)
#                 dof0 -= 1
#                 if len(hot) == max_hits:
#                     break
#     return hot
# dof_vars = find_dof_variables(model)
# print("Le variabili che fanno DoF:")
# for v in dof_vars:
#     print("  -", v.name)


In [ ]:
# from pyomo.environ import Var, value

# def list_unfixed_variables(model, max_lines=200, stream=None):
#     """
#     Stampa le prime `max_lines` variabili non fissate di `model`.
#     Funziona con Pyomo / IDAES di qualunque versione.
#     """
#     import sys
#     if stream is None:
#         stream = sys.stdout

#     # raccolgo tutte le Var non fissate (scorro anche nei blocchi figli)
#     unfixed = [
#         v for v in model.component_data_objects(Var, descend_into=True)
#         if not v.fixed
#     ]
#     unfixed.sort(key=lambda v: v.name)           # per avere ordine stabile

#     stream.write(
#         f"\nUNFIXED VARIABLES  (tot = {len(unfixed)})\n"
#         + "-"*40 + "\n"
#     )

#     for i, v in enumerate(unfixed):
#         if i == max_lines:
#             stream.write(f"... (altri {len(unfixed)-max_lines} omessi)\n")
#             break
#         # value() prova a valutare la variabile se ha un valore (altrimenti 'None')
#         try:
#             val = value(v)
#             stream.write(f"{v.name:65s}  {val:12.6g}\n")
#         except:
#             stream.write(f"{v.name:65s}  (no value)\n")

# list_unfixed_variables(model, max_lines=500)



In [ ]:
from pyomo.util.infeasible import log_infeasible_constraints

iscale.calculate_scaling_factors(model)
model.fs.sofc.initialize(optarg={"max_iter":30})
print("\n----- MODEL STATISTICS -----")
print(report_statistics(model))

solver = pyo.SolverFactory("ipopt")
solver.options = {"tol": 1e-6, "max_iter": 1_000, "print_level": 0}

results = solver.solve(model, tee=True)
log_infeasible_constraints(model, log_expression=True)


NameError: name 'model' is not defined

In [ ]:
from idaes.core.util.model_statistics import (
    degrees_of_freedom, 
    report_statistics,
    activated_constraints_set,
    activated_equalities_set,
    unfixed_variables_set,
    fixed_variables_set,
    variables_set,
)

from pyomo.environ import value

def report_model_dof(model, stream_label, stream):
    separator = '=' * 60
    subsection = '-' * 60

    def format_var(i, var):
        try:
            val = value(var)
            val_str = f"{val: .6e}" if abs(val) < 1e5 else f"{val: .3f}"
        except:
            val_str = "[NaN]"
        return f"{i:>3} - {val_str:>14}   {var}"

    print(f"\n{separator}")
    print(f"{stream_label:^60}")
    print(f"{separator}")

    print("\nSTREAM PROPERTIES:")
    print(f"{'Flow [mol/s]':<20}: {value(stream.flow_mol[0]):.6e}")
    print(f"{'Temperature [K]':<20}: {value(stream.temperature[0]):.2f}")
    print(f"{'Pressure [Pa]':<20}: {value(stream.pressure[0]):.2f}")

    print("\nMOLE FRACTIONS:")
    for t, c in stream.mole_frac_comp.keys():
        print(f"  x_{c:<10}: {value(stream.mole_frac_comp[t, c]):.6e}")

    print(f"\nDOF after {stream_label}: {degrees_of_freedom(model)}")

    # print(f"\n{subsection}")
    # print("MODEL STATISTICS REPORT:")
    # print(subsection)
    # report_statistics(model)

    # print(f"\n{subsection}")
    # print("VARIABLES:")
    # print(subsection)
    # for i, v in enumerate(variables_set(model)):
    #     print(format_var(i + 1, v))

    # print(f"\n{subsection}")
    # print("ACTIVATED CONSTRAINTS:")
    # print(subsection)
    # for i, c in enumerate(activated_constraints_set(model)):
    #     print(f"{i + 1:>3} - {c}")

    # print(f"\n{subsection}")
    # print("ACTIVATED EQUALITIES:")
    # print(subsection)
    # for i, eq in enumerate(activated_equalities_set(model)):
    #     print(f"{i + 1:>3} - {eq}")

    # print(f"\n{subsection}")
    # print("UNFIXED VARIABLES:")
    # print(subsection)
    # for i, ufv in enumerate(unfixed_variables_set(model)):
    #     print(format_var(i + 1, ufv))

    # print(f"\n{subsection}")
    # print("FIXED VARIABLES:")
    # print(subsection)
    # for i, fv in enumerate(fixed_variables_set(model)):
    #     print(format_var(i + 1, fv))

In [ ]:
# Esempio di richiamo strutturato per tutte le unità e i flussi associati

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_h2, "Translator H2 - Inlet", model.fs.sofc.translator_h2.inlet)
report_model_dof(model.fs.sofc.translator_h2, "Translator H2 - Outlet", model.fs.sofc.translator_h2.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator Air (O2 Rich e O2 Poor)
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.separator, "Separator Air - Inlet", model.fs.sofc.separator.inlet)
report_model_dof(model.fs.sofc.separator, "Separator Air - O2 Rich", model.fs.sofc.separator.o2_rich_strm)
report_model_dof(model.fs.sofc.separator, "Separator Air - O2 Poor", model.fs.sofc.separator.o2_poor_strm)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2-rich da separator a reaction_props
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_o2, "Translator O2 - Inlet", model.fs.sofc.translator_o2.inlet)
report_model_dof(model.fs.sofc.translator_o2, "Translator O2 - Outlet", model.fs.sofc.translator_o2.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per H2 e O2 inlet
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.mixer, "Mixer - O2 Inlet", model.fs.sofc.mixer.o2_rich_strm)
report_model_dof(model.fs.sofc.mixer, "Mixer - H2 Inlet", model.fs.sofc.mixer.fuel_strm)
report_model_dof(model.fs.sofc.mixer, "Mixer - Mixed Outlet", model.fs.sofc.mixer.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Stoichiometric Reactor
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.reactor, "Reactor - Inlet", model.fs.sofc.reactor.inlet)
report_model_dof(model.fs.sofc.reactor, "Reactor - Outlet", model.fs.sofc.reactor.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Separator water e O2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.reactor_separator, "Separator Reactor - O2 Stream", model.fs.sofc.reactor_separator.o2_strm)
report_model_dof(model.fs.sofc.reactor_separator, "Separator Reactor - Water Stream", model.fs.sofc.reactor_separator.water_strm)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator H2O e H2
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_h2_out, "Translator H2 Out - Inlet", model.fs.sofc.translator_h2_out.inlet)
report_model_dof(model.fs.sofc.translator_h2_out, "Translator H2 Out - Outlet", model.fs.sofc.translator_h2_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Translator O2-rich → air
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.translator_o2_out, "Translator O2 Out - Inlet", model.fs.sofc.translator_o2_out.inlet)
report_model_dof(model.fs.sofc.translator_o2_out, "Translator O2 Out - Outlet", model.fs.sofc.translator_o2_out.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Heater condizionamento finale
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.heater, "Heater - Inlet", model.fs.sofc.heater.inlet)
report_model_dof(model.fs.sofc.heater, "Heater - Outlet", model.fs.sofc.heater.outlet)

# ----------------------------------------------------------------------------------------------------------------
# UNITÀ: Mixer per O2 riciclato e N2 in uscita
# ----------------------------------------------------------------------------------------------------------------
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - O2 Inlet", model.fs.sofc.mixer_out.o2_strm_final)
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - N2 Inlet", model.fs.sofc.mixer_out.o2_poor_strm)
report_model_dof(model.fs.sofc.mixer_out, "Mixer Outlet - Mixed Outlet", model.fs.sofc.mixer_out.outlet)



                   Translator H2 - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_H2        : 9.900000e-01
  x_H2O       : 1.000000e-02

DOF after Translator H2 - Inlet: 0

                   Translator H2 - Outlet                   

STREAM PROPERTIES:
Flow [mol/s]        : 7.539700e-02
Temperature [K]     : 942.05
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_H2        : 9.900000e-01
  x_O2        : 1.430472e-15
  x_H2O       : 1.000000e-02

DOF after Translator H2 - Outlet: 0

                   Separator Air - Inlet                    

STREAM PROPERTIES:
Flow [mol/s]        : 2.000000e+00
Temperature [K]     : 948.45
Pressure [Pa]       : 101325.00

MOLE FRACTIONS:
  x_O2        : 2.100000e-01
  x_N2        : 7.900000e-01

DOF after Separator Air - Inlet: 0

                  Separator Air - O2 Rich                   

STREAM PROPERTIES:
Flow [mol/s]        :

In [ ]:
visualize(model.fs, "My Process Flowsheet")

2025-06-09 12:21:57 [INFO] idaes.idaes_ui.fv.fsvis: Using HTTP server on localhost, port 62018
2025-06-09 12:21:57 [INFO] idaes.idaes_ui.fv.fsvis: Loading saved flowsheet from 'My Process Flowsheet.json'
2025-06-09 12:21:57 [INFO] idaes.idaes_ui.fv.fsvis: Saving flowsheet to default file 'My Process Flowsheet.json' in current directory (c:\Users\Sara\Desktop\IDAES\SOFC)
2025-06-09 12:21:57 [WARNING] idaes.core.base.process_base: Could not serialize stream table: Unit model fs.sofc does not have the standard Port names (inlet and outlet). Please contact the unit model developer to develop a unit specific stream table.
2025-06-09 12:21:57 [WARNING] idaes.idaes_ui.fv.fsvis: Flowsheet name changed: old='My Process Flowsheet' new='My-Process-Flowsheet'
Flowsheet name changed to 'My-Process-Flowsheet'
2025-06-09 12:21:57 [INFO] idaes.idaes_ui.fv.fsvis: Flowsheet visualization at: http://localhost:62018/app?id=My-Process-Flowsheet


VisualizeResult(store=<idaes_ui.fv.persist.FileDataStore object at 0x000001C70206F4C0>, port=62018, server=<idaes_ui.fv.model_server.FlowsheetServer object at 0x000001C701F983D0>)